# 导入模块

In [ ]:
from selenium import webdriver
import time
import re
import os
import requests

# 创建数据库

In [ ]:
from pony.orm import *

db = Database()

class Ttimu(db.Entity):
    id = PrimaryKey(int, auto=True)
   lx = Optional(str)
    bh = PrimaryKey(str, auto=True)
    url = Optional(str)
    bt = Optional(str)
    fs = Optional(str)
    tgs = Optional(str)
    tjs = Optional(str)
    tgl = Optional(str)
    sj = Optional(str)
# db.generate_mapping()


# 数据库函数

In [ ]:
def create_database(dbpath):
    """创建数据库，如果数据库文件存在则删除之后再创建"""
    if os.path.exists(dbpath):
        os.remove(dbpath)
    f = open(dbpath,"w")
    f.close() 
    db.bind(provider='sqlite', filename=dbpath)
    db.generate_mapping(create_tables=True)
    set_sql_debug(True)


# 定义获取网页中的题型函数

In [ ]:
def extract_list_info(url,category):
    html = get_html(url)
    result = []
    reobj = re.compile(r'<tr><td class="answerIcon_1du7d"></td><td>([\d\D]*?)</td><td><a href="([\d\D]*?)" class="">([\d\D]*?)</a></td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td></tr>')
    for match in reobj.finditer(html):
        biaohao = match.group(1)
        lianjie = "https://pintia.cn/" + match.group(2)
        biaoti = match.group(3)
        fenshu = match.group(4)
        tongguoshu = match.group(5)
        tijiaosshu = match.group(6)
        tongguolv = match.group(7)
        for a,b,c,d,e,f,g in zip(biaohao,lianjie,biaoti,fenshu,tongguoshu,tijiaoshu,tongguolv):
            result.append((category,a,b,c,d,e,f,g))
    return result

# 定义获取题目详情页html代码

In [ ]:
def get_html(url):
    driver=webdriver.Chrome()
    driver.get(url)
    time.sleep(2)
    h=driver.page_source
    timu.html = h
    db.commit()
    driver.close()

# 定义获取题目内容

In [ ]:
def get_content(html):
    if re.search(r'<div>[\d\D]*?<div class="ques-view"><p>([\d\D]*?)</p>', html):
        match = re.search(r'<div>[\d\D]*?<div class="ques-view"><p>([\d\D]*?)</p>', html)
        tmunr = match.group(1).strip()
        timu.content = tmunr
        db.commit()


# 总合最后一步

In [ ]:
get_create_database()

# 采集编程题

In [ ]:
url = "https://pintia.cn/problem-sets/1111652100718116864/problems/type/7"
driver=webdriver.Chrome()
driver.get(url)
time.sleep(2)
h=driver.page_source
Ttimu.html = h
time.sleep(2)
get_timudata(h)
time.sleep(2)
driver.close()


# 采集填空题

In [ ]:
url = "https://pintia.cn/problem-sets/1111652100718116864/problems/type/6"
driver=webdriver.Chrome()
driver.get(url)
time.sleep(2)
h=driver.page_source
Ttimu.html = h
time.sleep(2)
get_timudata(h)
driver.close()

In [ ]:
with db_session:
    for timu in Ttimu.select():
        get_html(timu.href)
        get_content(timu.html)